# PROYECTO 1

Encargado: Daniela Ricaurte - 201822966

# # 1.  Instalación e importanción de librerías.

### Instalación de liberías

In [1]:
!pip install contractions

In [2]:
!pip install inflect
!pip install pandas-profiling==2.7.1 

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/Meli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/Meli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/Meli/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Importación de librerías

In [6]:
import pandas as pd
import numpy as np
from nltk import pos_tag
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

import sys
from pandas_profiling import ProfileReport

import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix, plot_precision_recall_curve
from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt

## 2. Perfilamiento y entendimiento de los datos

In [7]:
np.random.seed(500)

### Lectura de datos

In [8]:
df = pd.read_csv("SuicidiosProyecto.csv",encoding='latin-1')

### Entendimiento de los datos

In [9]:
df.shape

(195700, 3)

In [10]:
display(df.sample(5))

,Unnamed: 0,text,class
132749,235806,Not interestedYou can tell me I have so much t...,suicide
182994,93385,"I'm 18, I get urges to hurt people and I want ...",suicide
178942,152563,And thus the moment of realization came I wast...,non-suicide
133692,83607,What do I do Finish my school work \n\nStay on...,non-suicide
128222,305149,Wish there was a way to kill myself without hu...,suicide


In [11]:
df.dtypes

Unnamed: 0     int64
text          object
class         object
dtype: object

In [12]:
df['class']. value_counts()

non-suicide    110165
suicide         85535
Name: class, dtype: int64

In [13]:
df['text'].isna().sum()

0

### Entendimiento de los Datos

In [14]:
#textos = df.copy()
#textos['Conteo'] = [len(x) for x in textos['text']]
#textos['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos['text']]
#textos['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos['text']]

# Se realiza un perfilamiento de los datos con la librería pandas profiling
#ProfileReport(textos)

## 3. Preparación de datos

### Limpieza de datos

In [15]:
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

In [16]:
 def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

In [17]:
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [18]:
def remove_stopwords_and_non_alpha(words):
    """Remove stopwords from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in set(stopwords.words('english')) and word.isalpha():
            new_words.append(word)
    return new_words

In [19]:
def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

In [20]:

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    stop_words = set(stopwords.words('english'))
    new_words = []
    for word in words:
        if word not in stop_words and word.isalpha(): 
            new_words.append(word)
    return new_words

In [21]:
def preprocessing(words):
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words

### Tokenización

In [22]:
df['text'] = df['text'].apply(contractions.fix)

In [23]:
df['words'] = df['text'].apply(word_tokenize).apply(preprocessing) #Aplica la eliminación del ruido
df.head()

,Unnamed: 0,text,class,words
0,173271,i want to destroy myselffor once everything wa...,suicide,"[want, destroy, myselffor, everything, startin..."
1,336321,I kind of got behind schedule with learning fo...,non-suicide,"[kind, got, behind, schedule, learning, next, ..."
2,256637,I am just not sure anymoreFirst and foremost: ...,suicide,"[sure, anymorefirst, foremost, brazil, judge, ..."
3,303772,please give me a reason to liveThats too much ...,suicide,"[please, give, reason, livethats, much, reason..."
4,293747,27f struggling to find meaning moving forwardI...,suicide,"[struggling, find, meaning, moving, forwardi, ..."


### Normalización

In [24]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [25]:
 def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer=PorterStemmer() 
    # Se usa porter stemmer debido a que es una de las Eliminación de Prefijos y Sufijos mayormente usados ya que no es agresivo
    # ya que es intuitivo e intuitivo.
    
    new_words = []
    for word in words:
        word_Final = str(stemmer.stem(word))
        new_words.append(word_Final)
    return new_words
   

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    
    new_words = []
    lemmatizer=WordNetLemmatizer()
    for word in words:
        word_Final = str(lemmatizer.lemmatize(word, pos='v'))
        new_words.append(word_Final)
    return new_words
    
    

def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems + lemmas

df['words'] = df['words'].apply(stem_and_lemmatize) #Aplica lematización y Eliminación de Prefijos y Sufijos.
df.head()

,Unnamed: 0,text,class,words
0,173271,i want to destroy myselffor once everything wa...,suicide,"[want, destroy, myselffor, everyth, start, fee..."
1,336321,I kind of got behind schedule with learning fo...,non-suicide,"[kind, got, behind, schedul, learn, next, week..."
2,256637,I am just not sure anymoreFirst and foremost: ...,suicide,"[sure, anymorefirst, foremost, brazil, judg, s..."
3,303772,please give me a reason to liveThats too much ...,suicide,"[pleas, give, reason, livethat, much, reason, ..."
4,293747,27f struggling to find meaning moving forwardI...,suicide,"[struggl, find, mean, move, forwardi, admit, b..."


In [26]:
df_new=df
df_new['words'] = df_new['words'].apply(lambda x: ' '.join(map(str, x)))
df_new

df_new.to_csv('SuicidiosProyectoFinal2.csv')

In [27]:
#dff = pd.read_csv("DatosSuicidio/SuicidiosProyectoFinal.csv",encoding='latin-1')

In [28]:
#dff['words'].apply(word_tokenize)

In [29]:
#dff.head()


In [30]:
#dff.dtypes

### Set de entreno y Set de prueba

In [31]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['words'],df['class'],test_size=0.3)

### Codificación de los datos

In [32]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [33]:
Train_X

117913    friend go take go buy sex toy good friend thin...
62660     rememb time want dieth titl say realli pray di...
9276      burden societyi disabl work reli tax tax evil ...
49651     anyon chat iam desper bore convers wide variet...
136070    imagin think hairstyl anim cross racist fuck h...
                                ...                        
47928     yup definit amp process img heheh yup definite...
101131    want diei birthday come realiz get older noth ...
16562     feel worst want overi feel like idiot continu ...
72186     hate actual consid iti want yknow know futur n...
34858     want end alliam useless piec shit want end all...
Name: words, Length: 136990, dtype: object

### Vectorización de palabras
Metodo usado: TF-IDF

In [34]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df['words'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [35]:
print(Tfidf_vect.vocabulary_)

{'want': 4807, 'destroy': 1203, 'everyth': 1532, 'start': 4201, 'feel': 1668, 'okay': 3079, 'came': 664, 'donat': 1309, 'know': 2491, 'use': 4719, 'cope': 961, 'reason': 3601, 'iam': 2204, 'tear': 4396, 'skin': 4025, 'shred': 3969, 'swallow': 4350, 'everi': 1523, 'pill': 3289, 'find': 1706, 'right': 3737, 'alon': 158, 'room': 3756, 'wall': 4802, 'slowli': 4051, 'fall': 1621, 'ita': 2409, 'matter': 2736, 'time': 4506, 'snap': 4070, 'final': 1698, 'end': 1456, 'everything': 1533, 'come': 855, 'every': 1524, 'alone': 159, 'slowly': 4052, 'itas': 2413, 'finally': 1699, 'kind': 2474, 'got': 1916, 'behind': 452, 'schedul': 3826, 'learn': 2543, 'next': 2974, 'week': 4841, 'test': 4430, 'alreadi': 163, 'studi': 4269, 'good': 1909, 'minim': 2830, 'anyth': 227, 'yet': 4974, 'still': 4220, 'day': 1106, 'option': 3108, 'pull': 3506, 'tell': 4412, 'parent': 3183, 'freak': 1795, 'possibl': 3369, 'super': 4317, 'hard': 2021, 'get': 1883, 'schedule': 3827, 'already': 164, 'study': 4271, 'anything': 22

In [36]:
print(Train_X_Tfidf)

  (0, 4562)	0.45093793146870187
  (0, 4471)	0.14678960756934684
  (0, 4372)	0.1798729725032422
  (0, 4315)	0.4313119747617264
  (0, 3915)	0.32297113944546607
  (0, 1909)	0.1898892323464896
  (0, 1899)	0.38579004740347367
  (0, 1809)	0.332392612253166
  (0, 817)	0.2574855377712115
  (0, 655)	0.30122682196166306
  (1, 4968)	0.09718812086473229
  (1, 4966)	0.08523539611971126
  (1, 4946)	0.1449203335060185
  (1, 4939)	0.05360302562568395
  (1, 4911)	0.16080926051662991
  (1, 4853)	0.0756226819238418
  (1, 4807)	0.08479103471981013
  (1, 4801)	0.09629198209677929
  (1, 4610)	0.02855896304511645
  (1, 4593)	0.028556913229590884
  (1, 4516)	0.05826639810033224
  (1, 4515)	0.058280736289481876
  (1, 4506)	0.052534420872708244
  (1, 4485)	0.1059282999608131
  (1, 4471)	0.09879111659295678
  :	:
  (136988, 1843)	0.10822852943441784
  (136988, 1699)	0.10113959546450281
  (136988, 1698)	0.0964265503465987
  (136988, 1668)	0.11057301313672795
  (136988, 1517)	0.1272210219866166
  (136988, 1474)	0.

In [ ]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

In [ ]:
confusion_matrix(Test_Y, predictions_SVM)

In [ ]:
plot_confusion_matrix(SVM, Test_X, Test_Y)  
plt.show() 

In [ ]:
print(classification_report(Test_Y, predictions_SVM))

### Grid Search

In [ ]:
>>> from sklearn.model_selection import GridSearchCV
>>> parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)],
...               'tfidf__use_idf': (True, False),
...               'clf-svm__alpha': (1e-2, 1e-3),
... }
gs_clf_svm = GridSearchCV(SVM, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(twenty_train.data, twenty_train.target)
gs_clf_svm.best_score_
gs_clf_svm.best_params_